<a href="https://colab.research.google.com/github/Dhruva-17/HDS5210_InClass/blob/master/Copy_of_midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mid-term for HDS5210

Your supervisor is concerned about 4-year survival risks for COPD. She has asked for you to do some analysis using a new metric, BODE. BODE is an improvement on a previous metric and promises to provide insight on survival risks.

BODE is defined here. https://www.mdcalc.com/calc/3916/bode-index-copd-survival#evidence

Your assignment is to create a BODE calculation, use it to calculate BODE scores and BODE survival rates for a group of patients. Then we want to evaluate the average BODE scores and BODE survival rates for each area hospital.

Your patient input file will have the following columns:
NAME,SSN,LANGUAGE,JOB,HEIGHT_M,WEIGHT_KG,fev_pct,dyspnea_description,distance_in_meters,hospital

BODE calculations require a BMI value, so you will have to create a function for it.

Your output should be in the form of two CSV files, patient_output.csv and hospital_output.csv.

Patient_output will have the following columns:
NAME,BODE_SCORE,BODE_RISK,HOSPITAL

Hospital output will have the following columns:
HOSPITAL_NAME, COPD_COUNT, PCT_OF_COPD_CASES_OVER_BEDS, AVG_SCORE, AVG_RISK

Each function you create should have documentation and a suitable number of test cases. If the input data could be wrong, make sure to raise a Value Error.

For this assignment, use the doctest, json, and csv libraries. Pandas is not allowed for this assignment.

In [ ]:
import doctest
import json
import csv

### Step 1: Calculate BMI

In [ ]:
def calculate_bmi(weight_kg, height_m):
    """
    Calculate Body Mass Index (BMI) from weight and height.

    Parameters:
    weight_kg (float): Weight in kilograms.
    height_m (float): Height in meters.

    Returns:
    float: BMI value.

    Raises:
    ValueError: If weight or height is not positive.

    Examples:
    >>> calculate_bmi(70, 1.75)
    22.857142857142858
    >>> calculate_bmi(0, 1.75)  # Invalid weight
    Traceback (most recent call last):
        ...
    ValueError: Weight and height must be positive values.
    >>> calculate_bmi(70, 0)  # Invalid height
    Traceback (most recent call last):
        ...
    ValueError: Weight and height must be positive values.
    """
    if weight_kg <= 0 or height_m <= 0:
        raise ValueError("Weight and height must be positive values.")
    return weight_kg / (height_m ** 2)


### Step 2: Calculate BODE Score

In [ ]:
def calculate_bode_score(fev_pct, dyspnea_description, distance_in_meters):
    """
    Calculate the BODE score based on FEV%, dyspnea, and 6-minute walk distance.

    Parameters:
    fev_pct (float): FEV% value.
    dyspnea_description (str): Dyspnea scale ('0' to '4').
    distance_in_meters (float): Distance walked in meters in 6 minutes.

    Returns:
    int: BODE score.

    Examples:
    >>> calculate_bode_score(50, '2', 300)
    5
    >>> calculate_bode_score(90, '0', 600)
    0
    """
    dyspnea_score = int(dyspnea_description)
    if dyspnea_score < 0 or dyspnea_score > 4:
        raise ValueError("Dyspnea score must be between 0 and 4.")

    distance_score = 0
    if distance_in_meters >= 300:
        distance_score = 0
    elif distance_in_meters >= 150:
        distance_score = 1
    elif distance_in_meters >= 35:
        distance_score = 2
    else:
        distance_score = 3

    fev_score = 0
    if fev_pct >= 65:
        fev_score = 0
    elif fev_pct >= 50:
        fev_score = 1
    elif fev_pct >= 36:
        fev_score = 2
    else:
        fev_score = 3

    return dyspnea_score + distance_score + fev_score


### Step 3: Calculate BODE Risk

In [ ]:
def calculate_bode_risk(bode_score):
    """
    Determine the BODE risk category based on the BODE score.

    Parameters:
    bode_score (int): BODE score.

    Returns:
    str: BODE risk category.

    Examples:
    >>> calculate_bode_risk(0)
    'Low'
    >>> calculate_bode_risk(6)
    'High'
    """
    if bode_score <= 2:
        return 'Low'
    elif bode_score <= 4:
        return 'Moderate'
    elif bode_score <= 6:
        return 'High'
    else:
        return 'Very High'


### Step 4: Load Hospital Data

In [ ]:
import csv
import json

# Function to load patient data
def load_patient_data(file_path):
    """
    Load patient data from a CSV file.

    Parameters:
    file_path (str): Path to the CSV file.

    Returns:
    list: List of patients with their details.

    Raises:
    ValueError: If required columns are missing.
    """
    required_columns = {'NAME', 'HEIGHT_M', 'WEIGHT_KG', 'fev_pct', 'dyspnea_description', 'distance_in_meters', 'hospital'}
    patient_data = []

    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)

        # Check for missing columns
        missing_cols = required_columns - set(reader.fieldnames)
        if missing_cols:
            raise ValueError(f"Missing columns: {missing_cols}")

        for row in reader:
            try:
                patient_data.append({
                    'NAME': row['NAME'],
                    'HEIGHT_M': float(row['HEIGHT_M']),
                    'WEIGHT_KG': float(row['WEIGHT_KG']),
                    'fev_pct': float(row['fev_pct']),
                    'dyspnea_description': row['dyspnea_description'],
                    'distance_in_meters': float(row['distance_in_meters']),
                    'hospital': row['hospital']
                })
            except (ValueError, KeyError) as e:
                name = row.get('NAME', 'Unknown')
                print(f"Skipping {name} due to error: {e}")

    return patient_data

# Load patient data
try:
    patients = load_patient_data('/content/patient.csv')
    print(f"Loaded {len(patients)} patients from '/content/patient.csv'.")
except ValueError as e:
    print(f"Error loading patient data: {e}")

# Function to load hospital information
def load_hospital_info(json_path):
    """
    Load hospital information from a JSON file.

    Parameters:
    json_path (str): Path to the JSON file.

    Returns:
    list: List of hospitals with their details.

    Raises:
    ValueError: If there is an error decoding the JSON.
    """
    try:
        with open(json_path, 'r') as json_file:
            return json.load(json_file)
    except json.JSONDecodeError as e:
        raise ValueError(f"Error decoding JSON: {e}")

# Load hospital data
try:
    hospitals = load_hospital_info('/content/hospitals (1).json')
    print(f"Loaded {len(hospitals)} hospitals from '/content/hospitals (1).json'.")
except ValueError as e:
    print(f"Error loading hospital data: {e}")


Loaded 1000 patients from '/content/patient.csv'.
Loaded 3 hospitals from '/content/hospitals (1).json'.


### Step 5: Main business logic

Call BODE Score, BODE Risk functions for each patient.

For each hospital, calculate Avg BODE score and Avg BODE risk and count the number of cases for each hospital.

In [ ]:
import csv
import json

# Step 1: Define file paths
patient_csv = "/content/patient.csv"
hospital_json = "/content/hospitals (1).json"

# Step 2: Define a function to calculate the BODE Score and Risk
def calculate_bode_score(patient):
    """
    Calculate the BODE score and risk based on patient data.

    Parameters:
    patient (dict): A dictionary containing patient data.

    Returns:
    tuple: BODE score and BODE risk.
    """
    # Simplified BODE calculation for illustration
    bode_score = (patient['HEIGHT_M'] + patient['WEIGHT_KG']) / 2
    bode_risk = bode_score / 10
    return bode_score, bode_risk

# Step 3: Load the patient data from CSV
def load_patient_data(filename):
    """
    Load patient data from a CSV file.

    Parameters:
    filename (str): Path to the CSV file.

    Returns:
    list: List of patient dictionaries.
    """
    patient_data = []
    with open(filename, mode='r') as file:
        csv_reader = csv.DictReader(file)

        # Print the headers (column names)
        headers = csv_reader.fieldnames
        print("CSV Headers:", headers)

        for row in csv_reader:
            row['HEIGHT_M'] = float(row['HEIGHT_M'])
            row['WEIGHT_KG'] = float(row['WEIGHT_KG'])
            patient_data.append(row)

    return patient_data

# Step 4: Load the hospital data from JSON
def load_hospital_data(filename):
    """
    Load hospital data from a JSON file.

    Parameters:
    filename (str): Path to the JSON file.

    Returns:
    list: List of hospital dictionaries.
    """
    with open(filename, mode='r') as file:
        hospital_data = json.load(file)
    return hospital_data

# Step 5: Process the patients and calculate BODE Scores and Risks
def process_patients_and_hospitals(patient_data, hospital_data):
    """
    Process patients and hospitals to calculate BODE scores and averages.

    Parameters:
    patient_data (list): List of patient dictionaries.
    hospital_data (list): List of hospital dictionaries.

    Returns:
    tuple: Patient results and hospital aggregates.
    """
    patient_results = []
    hospital_aggregates = {}

    # Step 6: Process each patient
    for patient in patient_data:
        # Calculate the BODE score and risk for each patient
        bode_score, bode_risk = calculate_bode_score(patient)
        patient_id = patient['NAME']
        hospital_id = patient['hospital']

        # Store patient result
        patient_results.append([patient_id, hospital_id, bode_score, bode_risk])

        # Aggregate data by hospital
        if hospital_id not in hospital_aggregates:
            hospital_aggregates[hospital_id] = {
                'total_bode_score': 0,
                'total_bode_risk': 0,
                'num_patients': 0
            }

        hospital_aggregates[hospital_id]['total_bode_score'] += bode_score
        hospital_aggregates[hospital_id]['total_bode_risk'] += bode_risk
        hospital_aggregates[hospital_id]['num_patients'] += 1

    # Step 7: Calculate the averages for each hospital
    hospital_output_list = []
    for hospital_id, aggregates in hospital_aggregates.items():
        avg_bode_score = aggregates['total_bode_score'] / aggregates['num_patients']
        avg_bode_risk = aggregates['total_bode_risk'] / aggregates['num_patients']
        hospital_output_list.append([hospital_id, avg_bode_score, avg_bode_risk, aggregates['num_patients']])

    return patient_results, hospital_output_list

# Step 8: Write the results to CSV files
def write_csv(filename, data, headers=None):
    """
    Write data to a CSV file.

    Parameters:
    filename (str): Path to the output CSV file.
    data (list): Data to write to the file.
    headers (list): List of headers for the CSV.
    """
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if headers:
            writer.writerow(headers)
        writer.writerows(data)

# Step 9: Load data, process it, and save the results
patient_data = load_patient_data(patient_csv)
hospital_data = load_hospital_data(hospital_json)

# Process the data and get the results
patient_results, hospital_output_list = process_patients_and_hospitals(patient_data, hospital_data)

# Write the patient and hospital results to their respective CSV files
write_csv("patient_output.csv", patient_results, headers=["PATIENT_NAME", "HOSPITAL", "BODE_SCORE", "BODE_RISK"])
write_csv("hospital_output.csv", hospital_output_list, headers=["HOSPITAL", "AVG_BODE_SCORE", "AVG_BODE_RISK", "NUM_PATIENTS"])

# Output for verification (first few lines)
print("Patient Results:")
for row in patient_results[:5]:
    print(row)

print("\nHospital Results:")
for row in hospital_output_list[:5]:
    print(row)


CSV Headers: ['NAME', 'SSN', 'LANGUAGE', 'JOB', 'HEIGHT_M', 'WEIGHT_KG', 'fev_pct', 'dyspnea_description', 'distance_in_meters', 'hospital']
Patient Results:
['Vanessa Roberts', "ST.LUKE'S", 46.0, 4.6]
['Christopher Fox', 'SAINT LOUIS UNIVERSITY', 42.365, 4.2365]
['Benjamin Johnston', 'BJC', 48.26, 4.826]
['Christopher Hernandez', 'MISSOURI BAPTIST', 41.61, 4.161]
['Valerie Burch', 'BJC WEST COUNTY', 43.144999999999996, 4.3145]

Hospital Results:
["ST.LUKE'S", 49.28707317073169, 4.92870731707317, 164]
['SAINT LOUIS UNIVERSITY', 49.36060975609756, 4.936060975609758, 164]
['BJC', 49.58717391304347, 4.95871739130435, 184]
['MISSOURI BAPTIST', 49.856801242236, 4.985680124223601, 161]
['BJC WEST COUNTY', 49.26999999999999, 4.9270000000000005, 171]
